# 데이터 작업하기

In [1]:
import torch 
from torch import nn 
# 데이터셋을 반복 가능한 객체(iterable)화
from torch.utils.data import DataLoader
# sample과 label 저장 
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
""" 
torchvision.datasets 모듈 
: cifar, coco 등과 같은 다양한 실제 비전 데이터에 대한 Dataset을 포함

Fasion MNIST 데이터셋을 사용해보자.

이 튜토리얼에서는 Fasion MNIST 데이터셋을 사용한다.
모든 TorchVision dataset은 샘플과 정답을 
각각 변경하기 위한 transform과 target_transform의 두 인자를 포함한다.
"""

# # 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

C:\Users\JIYEON\anaconda3\envs\torch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
"""
Dataset을 DataLoader의 인자로 전달한다.
이는 데이터셋을 반복 가능한 객체(iterable)로 감싸고,
자동화된 배치, 샘플링, shuffle 및 
다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원한다.

여기서 배치 크기는 64로 정의한다.
즉, 데이터로더 객체의 각 요소는 64개의 특징과 정답을 묶음(batch)으로 반환한다.
"""

batch_size = 64

# 데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("shape of X [N, C, H, W] :", X.shape)
    print("Shape of y :", y.shape, y.dtype)
    break


shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


# 모델 만들기

In [48]:
"""
파이토치에서 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의
__init__함수에서 신경망의 계층을 정의하고
forward 함수에서 신경망에 데이터를 어떻게 전달할지 정한다.

가능한 경우 gpu로 신경망을 이동시켜 연산을 가속(accelerate)한다.

"""
# 학습에 사용할 cpu/gpu 장치 등록
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델 정의
class NeuralNetwork(nn.Module):
    # 신경망 계층 초기화 
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# NeuralNetwork의 인스턴스를 생성하고 
# 이를 device로 이동한 뒤, 
# 구조를 출력
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 신경망 모델 구성하기 - 자세히

In [32]:
"""
모델을 사용하기 위해 입력데이터 전달
이는 일부 백그라운드 연산들과 함께 모들의 forward를 실행
model.forward() 직접 호출하지 말기
--> 모델에 입력을 호출하면
각 클래스에 대한 raw 예측값이 있는 10차원 텐서가 반환됨
raw 예측값을 nn.Sotfmax 모듈의 인스턴스에 통과시켜
예측 확률을 얻는다.
"""

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([256], device='cuda:0')


In [50]:
# print(pred_probab)

### 모델 계층

In [34]:
"""
Fashion MNIST 모델의 계층들을 살펴보자.
이를 설명하기 위해 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와,
신경망을 통과할 때 어떤 일이 발생하는지 보자
"""

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [35]:
""" nn Flatten
nn.Flatten 계층을 초기화하여 
각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
dim=0의 미니배치 차원은 유지된다.
"""
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())


torch.Size([3, 784])


In [36]:
""" nn.Linear
선형 계층은 저장된 가중치와 편향을 사용하여 
입력에 선형 변환(linear transformation)을 적용하는 모듈"""
layer1 = nn.Linear(in_features=28*28, out_features=28)
hidden1 = layer1(flat_image)
print(hidden1.size())


torch.Size([3, 28])


In [37]:
"""nn.ReLU
비선형 activation은 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만든다.
비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고,
신경망이 다양한 현상을 학습할 수 있도록 돕는다."""
print(f"Before ReLu: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After Relu: {hidden1}")

Before ReLu: tensor([[-0.6363, -0.2606, -0.0175,  0.3945,  0.5081, -0.1264,  0.0226, -0.4330,
         -0.3636,  0.4240, -0.0865, -0.4544, -0.3437, -0.0628,  0.1276, -0.0256,
          0.2602, -0.0348,  0.1153,  0.0191, -0.4050, -0.4278, -0.1196, -0.0766,
          0.3642, -0.1131,  0.1299, -0.2919],
        [-0.1992, -0.3360,  0.1045,  0.4432,  0.4421,  0.1374,  0.1344, -0.4782,
         -0.0836,  0.2932, -0.0063, -0.3421, -0.1063,  0.2151,  0.0242,  0.0509,
         -0.3092, -0.0977, -0.0698,  0.0470, -0.1319, -0.6934,  0.1303, -0.0984,
          0.1728,  0.3856,  0.0517, -0.3452],
        [-0.3318, -0.1422, -0.0357,  0.0116,  0.7501, -0.0359,  0.0706, -0.2123,
         -0.2579,  0.3249,  0.2102, -0.0418, -0.3997,  0.3422, -0.0468, -0.2178,
          0.0534,  0.1922, -0.4640,  0.0874, -0.1810, -0.3468, -0.1928,  0.0176,
          0.3929,  0.2510,  0.0867, -0.2786]], grad_fn=<AddmmBackward>)


After Relu: tensor([[0.0000, 0.0000, 0.0000, 0.3945, 0.5081, 0.0000, 0.0226, 0.0000, 0.0000,

In [46]:
"""nn.Sequential 
: 순서를 갖는 모듈의 컨테이너
데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달된다.
순차 컨테이너를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있다.
 """

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(28,10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### 모델 매개변수

In [49]:
"""
신경망 내부의 많은 계층들은 매개변수화(parameterize) 된다.
즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.
nn.Module을 상속하여 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며,
모델의 parameters() 및 named_parameters() 메소드로 
모든 매개변수에 접근할 수 있게 된다. """
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values :{param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values :tensor([[ 0.0152,  0.0234,  0.0158,  ...,  0.0062,  0.0210,  0.0283],
        [ 0.0236,  0.0042, -0.0035,  ..., -0.0182, -0.0167,  0.0088]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values :tensor([-0.0090,  0.0249], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values :tensor([[ 0.0183,  0.0141,  0.0418,  ..., -0.0267, -0.0028, -0.0264],
        [-0.0421,  0.0430,  0.0350,  ..., -0.0058, -0.0168,  0.0283]],
       device='cuda:0'

# 모델 매개변수 최적화하기 

In [51]:
# 모델을 학습하려면 손실함수와 옵티마이저가 필요하다.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [52]:
# 각 학습 단계(training loop)에서 모델은 
# (배치로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 
# 예측 오류를 역전파하여 모델의 매개변수를 조정한다.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}    [{current:>5d}/{size:>5d}]")    


In [53]:
"""
모델이 학습하고 있는지 확인하기 위해
테스트 데이터셋으로 모델의 성능을 확인한다.
"""

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

        

In [57]:
"""
학습 단계는 여러번의 반복(에폭) 단계를 거쳐 수행된다.
각 에폭에서는 모델은 더 나은 예측을 하기 위해 매개변수를 학습한다.
각 에폭마다 모델의 정확도와 손실을 출력한다.
에폭마다 정확도가 증가하고 손실이 감소하는 것을 보려고 한다.
"""
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------
loss: 1.784283    [    0/60000]
loss: 1.699161    [ 6400/60000]
loss: 1.797474    [12800/60000]
loss: 1.764295    [19200/60000]
loss: 1.634640    [25600/60000]
loss: 1.918155    [32000/60000]
loss: 1.660005    [38400/60000]
loss: 1.895247    [44800/60000]
loss: 1.763144    [51200/60000]
loss: 1.704725    [57600/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 1.719640 

Epoch 2
-----------------------
loss: 1.757629    [    0/60000]
loss: 1.671656    [ 6400/60000]
loss: 1.771859    [12800/60000]
loss: 1.746646    [19200/60000]
loss: 1.612813    [25600/60000]
loss: 1.898856    [32000/60000]
loss: 1.640238    [38400/60000]
loss: 1.881477    [44800/60000]
loss: 1.742133    [51200/60000]
loss: 1.690654    [57600/60000]
Test Error: 
 Accuracy: 39.5%, Avg loss: 1.700365 

Epoch 3
-----------------------
loss: 1.733078    [    0/60000]
loss: 1.646955    [ 6400/60000]
loss: 1.748887    [12800/60000]
loss: 1.732134    [19200/60000]
loss: 1.595047    [25600/60000]


In [56]:
% nvidia-smi


UsageError: Line magic function `%` not found.


# 모델 저장하기 

In [58]:
"""
모델을 저장하는 일반적인 방법은 (모델의 매개변수들 포함)
내부 상태 사전(internal state dictionary)을 직렬화(serialize)하는 것
"""
torch.save(model.state_dict(), "./test.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


# 모델 불러오기

In [59]:
"""
모델을 불러오는 과정에는
1. 모델 구조를 다시 만들고
2. 상태 사전을 모델에 불러오는 과정
이 포함된다.
"""

model = NeuralNetwork()
model.load_state_dict(torch.load("test.pth"))

<All keys matched successfully>

In [64]:

"""
이제 모델을 사용하여 예측할 수 있다.
"""

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Sandal", Actual: "Ankle boot"
